In [1]:
import getpass
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
input_msg = input("請輸入問題")
result = model.invoke(
    [
        SystemMessage(content="請使用繁體中文回答。"),
        HumanMessage(content=input_msg),
    ]
)
display(Markdown(result.content))



e:\anaconda3\envs\AI\Lib\site-packages\langchain_google_genai\chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**人工智能（AI）** 是一種模擬人類智能的技術，讓電腦系統能夠執行通常需要人類智慧才能完成的任務。

**AI 的主要特徵包括：**

* **學習和適應：** AI 系統能夠從數據中學習，並根據新的信息調整其行為。
* **問題解決：** AI 系統能夠解決複雜的問題，並找到最佳的解決方案。
* **決策：** AI 系統能夠做出明智的決策，並根據數據和邏輯進行判斷。
* **自然語言處理：** AI 系統可以理解和生成人類語言，例如文本和語音。
* **視覺識別：** AI 系統可以識別和分析圖像和視頻。

**AI 的應用領域：**

* **自動駕駛汽車**
* **醫療診斷**
* **金融交易**
* **客戶服務**
* **遊戲開發**
* **機器人**
* **語音助理**

**AI 的類型：**

* **機器學習：** 讓電腦系統從數據中學習，而無需明確編程。
* **深度學習：** 是一種機器學習的子類別，使用人工神經網絡來處理大型數據集。
* **自然語言處理 (NLP)：** 讓電腦系統理解和生成人類語言。
* **電腦視覺：** 讓電腦系統識別和分析圖像和視頻。

**AI 的未來發展：**

* **更強大的計算能力**
* **更大量的數據**
* **更先進的算法**
* **更廣泛的應用**

總之，AI 是一種強大的技術，可以改變我們的生活和工作方式。它正在迅速發展，並將繼續在未來發揮越來越重要的作用。


In [2]:
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [3]:
## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
instruct_chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser() # stop the output

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print(zsc_call("Should I take the next exit, or keep going to the next one?"))

car


In [4]:
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State:  {'topic': 'boat'}


'The old, weathered boat, a seasoned traveler of the seas, whispered tales of forgotten islands and whispered secrets to the moonlit waves. \n'